In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure reproducibility for fair comparison, if desired
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(42)

DATA_DIR = '/home/kami/Documents/datasets/'  # Directory where the 'food-101' folder will be/is located
NUM_CLASSES = 101
NUM_EPOCHS = 3
BATCH_SIZE = 64  # Match the C++ version
LEARNING_RATE = 0.001

# Use more workers if your CPU has more cores to saturate the GPU pipeline
# Match the number of workers in your C++ DataLoader for a fair test
NUM_WORKERS = 8

Using device: cuda:0


In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Use Resize, not RandomResizedCrop, to match the C++ version
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading Food-101 dataset...")
# torchvision.datasets.Food101 handles downloading and parsing automatically
train_dataset = datasets.Food101(
    root=DATA_DIR,
    split='train',
    transform=data_transforms,
    download=False
)

# pin_memory=True is the Python equivalent of creating tensors in pinned memory
# It speeds up CPU-to-GPU data transfers
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


Loading Food-101 dataset...


In [10]:
shuffled_indices = list(iter(train_loader.sampler))

In [12]:
len(shuffled_indices)

75750